### Configuration
If the following variable is left blank then we will attempt to discover the correct URL from the notebook's environment

In [ ]:
GAFFER_REST_API_URL = ''

### Imports

In [ ]:
import io
import json
import pandas as pd
import os

!pip install plotly
import plotly.graph_objects as go

import gafferpy
from gafferpy import gaffer
from gafferpy import gaffer_connector

pd.set_option('display.max_rows', None)

### Set up connection to Gaffer REST API

In [ ]:
# Try to auto-detect REST API URL from environment
if not GAFFER_REST_API_URL:
    try:
        GAFFER_REST_API_URL = os.environ['GAFFER_REST_API_URL']
    except KeyError:
        pass

if not GAFFER_REST_API_URL:
    raise Exception('Please set GAFFER_REST_API_URL')
print('Using Gaffer REST API URL: {}'.format(GAFFER_REST_API_URL))

gc = gaffer_connector.GafferConnector(GAFFER_REST_API_URL)

### Get Graph Schema (from local file)

In [ ]:
import json
import glob
schema_from_file = {}
for schema_file in glob.glob('/etc/gaffer/schema/*.json'):
    with open(schema_file, 'r') as f:
        partial_schema = json.load(f)
        schema_from_file = {**schema_from_file, **partial_schema}
print(json.dumps(schema_from_file, indent=2))

### Get Graph Schema (from remote REST API)

In [ ]:
schema_response = gc.execute_get(gaffer.GetSchema())
schema = json.loads(schema_response)
print(json.dumps(schema, indent=2))

assert schema == schema_from_file, "Local schema does not match remote schema!"

### Verify that the graph looks to contain GB Road Traffic Counts dataset

In [ ]:
required_edge_types = ['RoadUse', 'RoadHasJunction', 'RegionContainsLocation', 'LocationContainsRoad', 'JunctionLocatedAt']
required_entity_types = ['Cardinality', 'JunctionUse']

for edge_type in required_edge_types:
    if edge_type not in schema['edges']:
        raise Exception('The graph schema does not contain an edge type called {}. Please ensure you are running this example notebook against a graph containing the GB Road Traffic Counts dataset.'.format(edge_type))

for entity_type in required_entity_types:
    if entity_type not in schema['entities']:
        raise Exception('The graph schema does not contain an entity type called {}. Please ensure you are running this example notebook against a graph containing the GB Road Traffic Counts dataset.'.format(entity_type))

### Get number of elements in graph

In [ ]:
gc.execute_operations(
    operations=[
        gaffer.GetAllElements(),
        gaffer.Count()
    ]
)

In [ ]:
edge_groups = gc.execute_operations(
    operations=[
        gaffer.GetAllElements(),
        gaffer.CountGroups()
    ]
)['edgeGroups']
edge_groups

### Find all junctions in the South West

In [ ]:
all_south_west_junctions = gaffer.JsonConverter.from_json(
    gc.execute_operation_chain(
        gaffer.OperationChain(
            operations=[
                gaffer.GetAdjacentIds(
                    input=[gaffer.EntitySeed(vertex="South West")],
                    view=gaffer.View(
                        edges=[
                            gaffer.ElementDefinition(
                                group="RegionContainsLocation"
                            )
                        ]
                    )
                ),
                gaffer.GetAdjacentIds(
                    view=gaffer.View(
                        edges=[
                            gaffer.ElementDefinition(
                                group="LocationContainsRoad"
                            )
                        ]
                    )
                ),
                gaffer.ToSet(),
                gaffer.GetAdjacentIds(
                    view=gaffer.View(
                        edges=[
                            gaffer.ElementDefinition(
                                group="RoadHasJunction"
                            )
                        ]
                    )
                )
            ]
        )
    )
)

all_south_west_junctions_dataframe = pd.DataFrame([entity_seed.vertex for entity_seed in all_south_west_junctions], columns=["junction"])
all_south_west_junctions_dataframe.sort_values(by='junction')

### Find busiest junctions in the South West in 2000

In [ ]:
busiest_south_west_junctions_in_2000 = gaffer.JsonConverter.from_json(
    gc.execute_operation_chain(
        gaffer.OperationChain(
            operations=[
                gaffer.GetElements(
                    input=all_south_west_junctions,
                    view=gaffer.View(
                        global_elements=[
                            gaffer.GlobalElementDefinition(
                                group_by=[]
                            )
                        ],
                        entities=[
                            gaffer.ElementDefinition(
                                group="JunctionUse",
                                pre_aggregation_filter_functions=[
                                    gaffer.PredicateContext(
                                        predicate=gaffer.InDateRangeDual(
                                            start="2000/01/01",
                                            end="2001/01/01"
                                        ),
                                        selection=[
                                            "startDate",
                                            "endDate"
                                        ]
                                    )
                                ]
                            )
                        ]
                    )
                ),
                gaffer.Sort(
                    comparators=[
                        gaffer.ElementPropertyComparator(
                            groups=["JunctionUse"],
                            property="count",
                            reversed=True
                        )
                    ],
                )
            ]
        )
    )
)

pd.DataFrame([[junction.vertex, junction.properties['count']['java.lang.Long']] for junction in busiest_south_west_junctions_in_2000], columns=["junction", "count"])

### Graph Car Counts for a Junction

In [ ]:
junction = 'M4:19'

response = gc.execute_operation_chain(
    gaffer.OperationChain(
        operations=[
            gaffer.GetElements(
                input=[gaffer.EntitySeed(junction)],
                view=gaffer.View(
                    entities=[
                        gaffer.ElementDefinition(
                            group="JunctionUse",
                            pre_aggregation_filter_functions=[
                                gaffer.PredicateContext(
                                    predicate=gaffer.InDateRangeDual(
                                        start="2000/01/01",
                                        end="2001/01/01"
                                    ),
                                    selection=[
                                        "startDate",
                                        "endDate"
                                    ]
                                )
                            ],
                            transient_properties={'CAR': 'Long'},
                            transform_functions=[
                                gaffer.FunctionContext(
                                    selection=["countByVehicleType"],
                                    function=gaffer.FreqMapExtractor(key="CAR"),
                                    projection=["CAR"]
                                )
                            ],
                            properties=["startDate", "endDate", "CAR"],
                        )
                    ]
                )
            ),
            gaffer.ToCsv(
                include_header=True,
                csv_generator=gaffer.CsvGenerator(
                    quoted=False,
                    comma_replacement=" ",
                    fields={'VERTEX': 'Junction', 'startDate': 'StartDate', 'CAR': 'CarCount'}
                )
            )
        ]
    )
)

df = pd.read_csv(io.StringIO("\n".join(response)))
fig = go.Figure(layout=go.Layout(title=go.layout.Title(text='{} Car Counts'.format(junction))), data=go.Bar(x=df['StartDate'], y=df['CarCount']))
fig.show()